# Code: _Looking for Lucy_

In [1]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
from scipy import stats

### Laod 3D data

In [2]:
mesh = o3d.io.read_triangle_mesh("../data/ship.ply")
tri = np.asarray(mesh.vertices)
x = tri[:, 0]
y = tri[:, 1]
z = tri[:, 2]
simplices = np.asarray(mesh.triangles)

### Figure 1.1

In [14]:
# The ship
fig = go.Figure(ff.create_trisurf(x=x, y=y, z=z,
                                  simplices=simplices,
                                  plot_edges=False,
                                  show_colorbar=False).data)
fig.update_traces(hoverinfo='none')

# The dots
fig.add_trace(
    go.Scatter3d(x=[-100], y=[0], z=[-60],
                 mode='markers',
                 hovertext='Unlikely',
                 hoverinfo='text',
                 showlegend=False)
)
fig.add_trace(
    go.Scatter3d(x=[0], y=[0], z=[-30],
                 mode='markers',
                 hovertext='Likely',
                 hoverinfo='text',
                 showlegend=False))

# Figure properties
camera = dict(
    eye=dict(x=-0.6, y=-1.2, z=0.2),
    center=dict(x=0.75, y=0, z=-0.4))

# Figure properties
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  height=400,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera)

In [12]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/ship.html',
               full_html=False,
               include_plotlyjs='cdn')

### Figure 1.2

In [5]:
# The ship
fig = go.Figure(ff.create_trisurf(x=x, y=y-2, z=z-118,
                                  simplices=simplices,
                                  plot_edges=False,
                                  show_colorbar=False).data)
fig.update_traces(hoverinfo='none')

# The bars
fig.add_trace(
    go.Scatter3d(x=[-100, -100, -100],
                 y=[-250, -250, -250],
                 z=[-166, -155, -146],
                 hovertext=["", "Ocean", ""],
                 hoverinfo='text',
                 mode='lines+text',
                 text=["", "<b>10%</b>", ""],
                 textposition="middle center",
                 textfont=dict(size=20, color='white'),
                 line=dict(width=120),
                 showlegend=False))

fig.add_trace(
    go.Scatter3d(x=[-100, -100, -100],
                 y=[0, 0, 0],
                 z=[-166, 100, 120],
                 hovertext=["", "Ship", ""],
                 hoverinfo='text',
                 mode='lines+text',
                 text=["", "<b>90%</b>", ""],
                 textposition="middle center",
                 textfont=dict(size=20, color='white'),
                 line=dict(width=120),
                 showlegend=False))

# The x axis
fig.add_trace(
    go.Scatter3d(x=[0, 0, 0],
                 y=[-350, 0, 270],
                 z=[-200, -200, -200],
                 line=dict(color='black', width=2),
                 mode='lines',
                 hoverinfo='none',
                 showlegend=False))


# The y axis
fig.add_trace(
    go.Scatter3d(x=[0, 0, 0],
                 y=[270, 270, 270],
                 z=[-200, 65, 200],
                 line=dict(color='black', width=2),
                 mode='lines',
                 hoverinfo='none',
                 showlegend=False))

# The y label
fig.update_layout(scene=dict(annotations=[dict(
                  showarrow=False,
                  x=0,
                  y=270,
                  z=0,
                  text="Probability",
                  xanchor="right",
                  font=dict(size=16),
                  textangle=-90)]))

# Viewpoint
camera = dict(
    eye=dict(x=-1.6, y=0, z=-0),
    center=dict(x=0, y=0, z=0))

# Figure properties
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  height=400,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,)

In [6]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/ship_bars.html',
               full_html=False,
               include_plotlyjs='cdn')

### Figure 2

In [7]:
# The ship
fig = go.Figure(ff.create_trisurf(x=x, y=y, z=z,
                                  simplices=simplices,
                                  plot_edges=False,
                                  show_colorbar=False).data)
fig.update_traces(hoverinfo='none')

# The x axis
fig.add_trace(
    go.Scatter3d(x=[0, 0, 0],
                 y=[-280, 0, 270],
                 z=[-80, -80, -80],
                 line=dict(color='black', width=2),
                 mode='lines',
                 hoverinfo='none',
                 showlegend=False))

annotations=[dict(
                showarrow=False,
                x=0,
                y=0,
                z=-80,
                yanchor='top',
                text=r'$e\mathrm{\ - \ Exact\ position}$',
                yshift=-10)]

# Vertical lines
fig.add_trace(
    go.Scatter3d(x=np.zeros(100),
                 y=np.ones(100) * 55,
                 z=np.linspace(-80, 135, 100),
                 line=dict(color='crimson', width=4, dash='dash'),
                 mode='lines',
                 hoverinfo='text',
                 hovertext='20m',
                 showlegend=False))

fig.add_trace(
    go.Scatter3d(x=np.zeros(100),
                 y=np.ones(100) * -55,
                 z=np.linspace(-80, 135, 100),
                 line=dict(color='crimson', width=4, dash='dash'),
                 mode='lines',
                 hoverinfo='text',
                 hovertext='30m',
                 showlegend=False))

# Text
annotations.append(dict(
                showarrow=False,
                x=0,
                y=0,
                z=80,
                text=r"$p(\mathrm{middle}\vert\mathrm{ship})\mathrm{\ -\ Probability}$",
                xanchor="right",
                textangle=-90))
fig.update_layout(scene=dict(annotations=annotations))

# Gaussian
fig.add_trace(
    go.Scatter3d(x=np.zeros(100),
                 y=np.linspace(-250, 250, 100),
                 z=stats.norm.pdf(np.linspace(-4, 4, 100), 0, 1) * 500,
                 line=dict(color='crimson', width=4),
                 hovertext='Lucy?',
                 mode='lines',
                 hoverinfo='text',
                 showlegend=False))

# Viewpoint
camera = dict(
    eye=dict(x=-1, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    projection=dict(type="orthographic"))

# Figure properties
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera)

In [8]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/ship_1dgauss.html',
               full_html=False,
               include_mathjax='cdn',
               include_plotlyjs='cdn')

### Figure 3

In [9]:
# The ship
fig = go.Figure(ff.create_trisurf(x=x, y=y, z=z-40,
                                  simplices=simplices,
                                  plot_edges=False,
                                  show_colorbar=False).data)
fig.update_traces(hoverinfo='none')

# Calculating multivariate Gaussian
X, Y = np.meshgrid(np.linspace(-1, 1, 25), np.linspace(-3, 3, 75))
mu = np.array([0., 0.])
Sigma = np.array([[0.1 , 0], [0,  0.4]])

pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

def multivariate_gaussian(pos, mu, Sigma):
    """Return the multivariate Gaussian distribution on array pos.

    pos is an array constructed by packing the meshed arrays of variables
    x_1, x_2, x_3, ..., x_k into its _last_ dimension.

    """

    n = mu.shape[0]
    Sigma_det = np.linalg.det(Sigma)
    Sigma_inv = np.linalg.inv(Sigma)
    N = np.sqrt((2*np.pi)**n * Sigma_det)
    fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

    return np.exp(-fac / 2) / N

Z = multivariate_gaussian(pos, mu, Sigma)

# Plotting the Gaussian
fig.add_trace(go.Surface(x=X*100,
                         y=Y*100,
                         z=Z*100,
                         contours_z=dict(show=True,
                                         usecolormap=True,
                                         highlightcolor="limegreen",
                                         project_z=True,
                                         start=10,
                                         end=70,
                                         size=10),
                         showscale=False,
                         hoverinfo='none'))

# Viewpoint
camera = dict(
    eye=dict(x=0, y=0, z=1.3),
    center=dict(x=0, y=0, z=0))

# Figure properties
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  height=350,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera)

In [10]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/ship_2dgauss.html',
               full_html=False,
               include_plotlyjs='cdn')